## selenium으로 데이터 가져오기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [2]:
driver = webdriver.Chrome("chromedriver.exe")

In [3]:
driver.get('https://www.youtube.com/channel/UCyn-K7rZLXjGl7VXGweIlcA/videos')

In [4]:
body = driver.find_element_by_tag_name('body')

In [5]:
# 스크롤 내리면서 탐색
SCROLL_PAUSE_TIME = 1
old_last_height = driver.execute_script('return document.documentElement.scrollHeight')
while True:
    body.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)
    last_height = driver.execute_script('return document.documentElement.scrollHeight')
    if last_height==old_last_height:
        break
    old_last_height = last_height

In [6]:
driver.execute_script('return document.documentElement.scrollHeight')

5918

In [7]:
dm_lst = driver.find_elements_by_id('dismissable')

In [8]:
# 유튜브 게시물에서 정보 받아오기
title_lst = []
time_lst = []
num_lst = []

for dm in dm_lst:
    title_lst.append(dm.find_element_by_id('video-title').text)
    
    ti = dm.find_element_by_id('overlays').text.split(':')
    if len(ti) == 3:
        sec = int(ti[0])*3600 + int(ti[1])*60 + int(ti[2])
    elif len(ti) == 2:
        sec = int(ti[0])*60 + int(ti[1])
    time_lst.append(sec)
    
    temp_num = dm.find_element_by_id('metadata-line').text[4:].split('\n')[0]
    if temp_num.find('만회') != -1:
        num = float(temp_num.replace('만회',''))*10000
    elif temp_num.find('억회') != -1:
        num = float(temp_num.replace('억회',''))*10000000
    elif temp_num.find('천회') != -1:
        num = float(temp_num.replace('천회',''))*1000
    elif temp_num.find('백회') != -1:
        num = float(temp_num.replace('백회',''))*100
    elif temp_num.find('회') != -1:
        num = float(temp_num.replace('회',''))
    num_lst.append(num)

## 판다스 이용해서 데이터 저장

In [9]:
import pandas as pd

In [10]:
dict_youtube={"title":title_lst, "video_time":time_lst, "view_num":num_lst}
youtube = pd.DataFrame(dict_youtube)
youtube.head()

,title,video_time,view_num
0,'우리 가게를 와본 사람보다 안 와본 사람이 훨씬 많아요.' ㅣ 백종원의 장사이야기,405,3600.0
1,정말 쉽게 만드는 마파두부!! ㅣ 백종원의 쿠킹로그,452,270000.0
2,맥주 안주로 딱! 매콤새콤 골뱅이무침! ㅣ 백종원의 백종원 레시피,467,210000.0
3,'쉽게 알고 덤볐는데 막상 해보니까 어려워' 그게 외식업이에요. ㅣ 백종원의 장사이야기,429,100000.0
4,"베이컨땡초김밥/게맛살마요김밥/돼지불고기김밥/무말랭이김밥, 나만의 김밥을 만들어 보세...",955,410000.0


In [11]:
# pandas로 csv 저장
youtube.to_csv("youtube.csv", mode='w', encoding='CP949', index=False)